In [286]:
import api_keys
import requests
import time as time
import pandas as pd
from bs4 import BeautifulSoup

In [259]:
client_id = api_keys.client_id
client_secret = api_keys.client_secret

In [381]:
class ArtsyScraper:
    
    def __init__(self, client_id, client_secret):
        
        auth_token_uri = 'https://api.artsy.net/api/tokens/xapp_token'

        api_url = 'https://api.artsy.net/api'

        token = requests.post(auth_token_uri, params={'client_id': client_id, 'client_secret': client_secret})
        self.token = token.json()['token']
        
        return None
    
    def scrape_api(self):
        self._api_scraper()
        
    def scrape_auctions(self):
        self.auction_results = self._auction_scraper()
        
    def make_slugs(self):
        self.artist_slugs = self.results_df.artist_slug.unique()

    def _api_scraper(self):
        time_split = 0.5
        
        self.results = []
        
        endpoint = api_url + '/artworks/'

        is_next = True
        
        keys_to_keep = ['id', 'slug', 'title', 'category', 'medium', 'date', 'iconicity', 'sold', 'image_versions']
        count = 0
        bad = 0
        while(is_next):
            try:
                r = requests.get(endpoint, headers={'X-Xapp-Token': self.token})

                rjson = r.json()

                artworks = rjson['_embedded']['artworks']

                for artwork in artworks:

                    artwork_result = {key: artwork[key] for key in artwork.keys() if key in keys_to_keep}
                    artwork_result['dimensions_height_in'] = artwork['dimensions']['in']['height']
                    artwork_result['dimensions_width_in'] = artwork['dimensions']['in']['width']
                    artwork_result['dimensions_depth_in'] = artwork['dimensions']['in']['depth']
                    artwork_result['permalink'] = artwork['_links']['permalink']['href']
                    artwork_result['api_link'] = artwork['_links']['self']['href']
                    artwork_result['thumb_link'] = artwork['_links']['thumbnail']['href']
                    artwork_result['image_link'] = artwork['_links']['image']['href']

                    is_next_genes = True
                    gene_api_url = artwork['_links']['genes']['href']
                    genes = []
                    while(is_next_genes):
                        genes_request = requests.get(gene_api_url, headers={'X-Xapp-Token': self.token})
                        for gene in genes_request.json()['_embedded']['genes']:
                            genes.append(gene['name'])
                        try:
                            gene_api_url = genes_request.json()['_links']['next']['href']
                            time.sleep(time_split)
                        except:
                            is_next_genes = False

                    artwork_result['genes'] = genes
                    artist_href = artwork['_links']['artists']['href']
                    artist_request = requests.get(artist_href, headers={'X-Xapp-Token': self.token}, params={'total_count':1})
                    artist_json = artist_request.json()['_embedded']['artists'][0]
                    artwork_result['artist'] = artist_json['name']
                    artwork_result['artist_slug'] = artist_json['slug']
                    artwork_result['artist_id'] = artist_json['id']

                    self.results.append(artwork_result)
                    print(f'{count+1} works collected. {bad} bad scrapes', end='\r')
                    count+=1

                    time.sleep(time_split)
            except:
                bad += 1
            try:
                next_url = rjson['_links']['next']
            except:
                is_next = False
                break
            
        return self.results
        
    def _auction_scraper(self):
        if self.artist_slugs != None:  
            pass
        else:
            print('need to run scrape()!')
            
            
            
        
    


In [382]:
scraper = ArtsyScraper(client_id, client_secret)

In [ ]:
scraper.scrape_api()

In [333]:
auction_results = []

for artist in results_df.artist_slug:
    url = f'https://www.artsy.net/artist/{artist}/auction-results?metric=in'
    
    auction_request = requests.get(url, headers={'X-Xapp-Token': token})
    time.sleep(.2)
    
    soup = BeautifulSoup(auction_request.content)
    script_string = soup.find_all('script')[-2].text.replace('\n              var __RELAY_BOOTSTRAP__ = ', '').replace(';\n            ','')

    auction_string = json.loads(script_string)
    auction_string = json.loads(auction_string)
    auction_string
    nodes = auction_string[1][1]['data']['artist']['auctionResultsConnection']['edges']
    
    for node in nodes:
        result = node['node']
        result['artist_slug'] = artist
        auction_results.append(result)
        time.sleep(.2)
    
    break


In [272]:
results_df = pd.DataFrame(scraper.results)

In [273]:
results_df.artist_slug.unique()

array(['gustav-klimt', 'francisco-de-goya', 'rembrandt-van-rijn',
       'leonardo-da-vinci', 'edouard-manet'], dtype=object)

In [132]:
import json

string_ = "[[\"{\\\"queryID\\\":\\\"artistRoutes_TopLevelQuery\\\",\\\"variables\\\":{\\\"artistID\\\":\\\"gustav-klimt\\\"}}\",{\"ok\":true,\"status\":200,\"url\":\"https:\\u002F\\u002Fmetaphysics-production.artsy.net\\u002Fv2\",\"headers\":{\"_headers\":{\"date\":[\"Fri, 29 Jul 2022 20:03:31 GMT\"],\"content-type\":[\"application\\u002Fjson; charset=utf-8\"],\"transfer-encoding\":[\"chunked\"],\"connection\":[\"close\"],\"x-powered-by\":[\"Express\"],\"access-control-allow-origin\":[\"*\"],\"etag\":[\"W\\u002F\\\"3af0-pe6whua1xs11KaR6d\\u002F2W+Qhz830\\\"\"],\"vary\":[\"Accept-Encoding\"],\"content-encoding\":[\"gzip\"],\"strict-transport-security\":[\"max-age=15724800; includeSubDomains\"]}},\"json\":{\"data\":{\"artist\":{\"slug\":\"gustav-klimt\",\"statuses\":{\"shows\":true,\"cv\":true,\"articles\":true,\"artworks\":true,\"auctionLots\":true},\"counts\":{\"forSaleArtworks\":186,\"auctionResults\":1351,\"artworks\":247,\"follows\":20784},\"related\":{\"genes\":{\"edges\":[{\"node\":{\"slug\":\"vienna-secession\",\"id\":\"R2VuZTo0ZGU2ODcxZWFkNTA2ZTAwMDEwMDU0ZWI=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"symbolism\",\"id\":\"R2VuZTo0ZDkwZDE5M2RjZGQ1ZjQ0YTUwMDAwNzI=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"art-nouveau\",\"id\":\"R2VuZTo0ZDkwZDE4ZmRjZGQ1ZjQ0YTUwMDAwMmU=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"engaged-with-byzantine-and-medieval-art\",\"id\":\"R2VuZTo1NmMzMzc0ODY0Mjg3MDc5ODMwMDAyMzY=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"modern-and-impressionist-art\",\"id\":\"R2VuZTo1MDM1NjU3NGFiNzQ5ODAwMDIwMDAwMDY=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"austria\",\"id\":\"R2VuZTo1MTgyYmVhMzRiNzJjOGYyMjMwMDAwNWQ=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"mixed-media\",\"id\":\"R2VuZTo0ZDkwZDE4ZWRjZGQ1ZjQ0YTUwMDAwMGY=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"work-on-paper\",\"id\":\"R2VuZTo0ZDkwZDE4ZmRjZGQ1ZjQ0YTUwMDAwMTY=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"painting\",\"id\":\"R2VuZTo0ZDkwZDE4ZWRjZGQ1ZjQ0YTUwMDAwMTA=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"patterns\",\"id\":\"R2VuZTo0ZTVmZmViZTFiYjkxNzAwMDEwMDE1NjY=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"individual-portrait\",\"id\":\"R2VuZTo0ZTU3MDhjYjVkY2ExNDAwMDEwMDNhYTM=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"erotic\",\"id\":\"R2VuZTo0ZGNkZjM0ZjgwODBhZDAwMDEwMDNhODM=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"human-figure\",\"id\":\"R2VuZTo0ZDkwZDE5M2RjZGQ1ZjQ0YTUwMDAwN2M=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"nude\",\"id\":\"R2VuZTo0ZDkzOTZkYjE3Y2IxMzI1MzcwMDA0ODA=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"landscapes\",\"id\":\"R2VuZTo0ZDkwZDE5M2RjZGQ1ZjQ0YTUwMDAwN2Q=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"flora\",\"id\":\"R2VuZTo0ZDkwZDE5NmRjZGQ1ZjQ0YTUwMDAwY2I=\",\"__typename\":\"Gene\"}}]}},\"highlights\":{\"artistPartnersConnection\":{\"edges\":[]},\"partnersConnection\":{\"edges\":[]}},\"insights\":[{\"type\":\"SOLO_SHOW\",\"__typename\":\"ArtistInsight\"},{\"type\":\"GROUP_SHOW\",\"__typename\":\"ArtistInsight\"},{\"type\":\"COLLECTED\",\"__typename\":\"ArtistInsight\"},{\"type\":\"REVIEWED\",\"__typename\":\"ArtistInsight\"},{\"type\":\"BIENNIAL\",\"__typename\":\"ArtistInsight\"},{\"type\":\"ACTIVE_SECONDARY_MARKET\",\"__typename\":\"ArtistInsight\"}],\"biographyBlurb\":{\"text\":\"\\u003Cp\\u003EGustav Klimt is best known for his opulently gilded, Byzantine-inspired \\u003Ca href=\\\"https:\\u002F\\u002Fwww.artsy.net\\u002Fgene\\u002Fart-nouveau\\\"\\u003Eart nouveau\\u003C\\u002Fa\\u003E  \\u003Ca href=\\\"https:\\u002F\\u002Fwww.artsy.net\\u002Fcollection\\u002Fgustav-klimt-portraits\\\"\\u003Eportraits\\u003C\\u002Fa\\u003E of women, which epitomize fin de siècle (and early 20th-century) Vienna. His most famous paintings, \\u003Cem\\u003EPortrait of Adele Bloch-Bauer\\u003C\\u002Fem\\u003E (1907) and \\u003Cem\\u003E\\u003Ca href=\\\"https:\\u002F\\u002Fwww.artsy.net\\u002Fartwork\\u002Fgustav-klimt-the-kiss\\\"\\u003EThe Kiss\\u003C\\u002Fa\\u003E\\u003C\\u002Fem\\u003E (1907), are among the most recognizable works of the early 20th century. Klimt’s \\u003Ca href=\\\"https:\\u002F\\u002Fwww.artsy.net\\u002Fgene\\u002Fsymbolism\\\"\\u003ESymbolist\\u003C\\u002Fa\\u003E pastiches of pale nudes, allegorical gardens, and erotic subject matter celebrated the intersection of fine and decorative art and became exemplars of the \\u003Ca href=\\\"http:\\u002F\\u002Fartsy.net\\u002Fgene\\u002Fvienna-secession\\\"\\u003EVienna Secession\\u003C\\u002Fa\\u003E. Today, his paintings, drawings, and objets belong in collections worldwide, including the Neue Galerie and the \\u003Ca href=\\\"https:\\u002F\\u002Fwww.artsy.net\\u002Fpartner\\u002Fthe-museum-of-modern-art\\\"\\u003EMuseum of Modern Art\\u003C\\u002Fa\\u003E in New York, and the Leopold Museum and Belvedere Museum and Palace in Vienna. On the secondary market, Klimt’s work routinely sells for millions. While Klimt’s narrative paintings explore psychosexual anxiety and the human condition, his vibrant landscapes demonstrate a keen engagement with light and the abstract patterns of nature. \\u003C\\u002Fp\\u003E\\n\",\"credit\":null,\"partnerID\":null},\"name\":\"Gustav Klimt\",\"nationality\":\"Austrian\",\"birthday\":\"1862\",\"deathday\":\"1918\",\"gender\":\"male\",\"href\":\"\\u002Fartist\\u002Fgustav-klimt\",\"meta\":{\"description\":\"Find the latest shows, biography, and artworks for sale by Gustav Klimt. Gustav Klimt is best known for his opulently gilded, Byzantine-inspired art nouveau …\"},\"alternate_names\":[],\"image\":{\"versions\":[\"four_thirds\",\"large\",\"square\",\"tall\"],\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002Fn3RTu8EOaXrq3UCBXjh1Ag\\u002Flarge.jpg\",\"square\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002Fn3RTu8EOaXrq3UCBXjh1Ag\\u002Fsquare.jpg\",\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fn3RTu8EOaXrq3UCBXjh1Ag%2Flarge.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fn3RTu8EOaXrq3UCBXjh1Ag%2Flarge.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fn3RTu8EOaXrq3UCBXjh1Ag%2Flarge.jpg 2x\"}},\"blurb\":\"Gustav Klimt is best known for his opulently gilded, Byzantine-inspired [art nouveau](https:\\u002F\\u002Fwww.artsy.net\\u002Fgene\\u002Fart-nouveau)  [portraits](https:\\u002F\\u002Fwww.artsy.net\\u002Fcollection\\u002Fgustav-klimt-portraits) of women, which epitomize fin de siècle (and early 20th-century) Vienna. His most famous paintings, _Portrait of Adele Bloch-Bauer_ (1907) and _[The Kiss](https:\\u002F\\u002Fwww.artsy.net\\u002Fartwork\\u002Fgustav-klimt-the-kiss)_ (1907), are among the most recognizable works of the early 20th century. Klimt’s [Symbolist](https:\\u002F\\u002Fwww.artsy.net\\u002Fgene\\u002Fsymbolism) pastiches of pale nudes, allegorical gardens, and erotic subject matter celebrated the intersection of fine and decorative art and became exemplars of the [Vienna Secession](http:\\u002F\\u002Fartsy.net\\u002Fgene\\u002Fvienna-secession). Today, his paintings, drawings, and objets belong in collections worldwide, including the Neue Galerie and the [Museum of Modern Art](https:\\u002F\\u002Fwww.artsy.net\\u002Fpartner\\u002Fthe-museum-of-modern-art) in New York, and the Leopold Museum and Belvedere Museum and Palace in Vienna. On the secondary market, Klimt’s work routinely sells for millions. While Klimt’s narrative paintings explore psychosexual anxiety and the human condition, his vibrant landscapes demonstrate a keen engagement with light and the abstract patterns of nature. \",\"artworks_connection\":{\"edges\":[{\"node\":{\"title\":\"Danaë [Das Werk Gustav Klimts]\",\"date\":\"1908-1914\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-danae-das-werk-gustav-klimts\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FStnM65TItGlWf869shn_lw\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FStnM65TItGlWf869shn_lw\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1OTYxNGE3Y2M5ZGMyNDZlYTBjOWE1ZjM=\"}},{\"node\":{\"title\":\"Judith [Das Werk Gustav Klimts]\",\"date\":\"1908-1914\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-judith-das-werk-gustav-klimts\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002F66PZrg542AFfN1LCo2JMvQ\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002F66PZrg542AFfN1LCo2JMvQ\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1Nzc3ZGMyNWNkNTMwZTY1ZjEwMDAxZjQ=\"}},{\"node\":{\"title\":\"Malcesine on Lake Garda [Gustav Klimt An Aftermath]\",\"date\":\"1931\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-malcesine-on-lake-garda-gustav-klimt-an-aftermath\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FlUJcL_7hQd8czIMBDLLK3Q\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FlUJcL_7hQd8czIMBDLLK3Q\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1NjZmNDQ5YzRjMGFmNTEzZDMwMDAwMzI=\"}},{\"node\":{\"title\":\"The Virgin [Das Werk Gustav Klimts]\",\"date\":\"1908-1914\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-the-virgin-das-werk-gustav-klimts\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FYme8vYIcH2W1BA_jqx0HhQ\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FYme8vYIcH2W1BA_jqx0HhQ\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1OTYxNGE3ZGIyMDJhMzRkNjY1MzIxYzE=\"}},{\"node\":{\"title\":\"Sunflowers [Gustav Klimt An Aftermath]\",\"date\":\"1931\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-sunflowers-gustav-klimt-an-aftermath\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FqlOeKMCvHLo_QtyznsR7Ug\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FqlOeKMCvHLo_QtyznsR7Ug\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1NjZmNDNmODc2MTQzZjdhNGUwMDAwMjA=\"}},{\"node\":{\"title\":\"Standing Nude [Fünfundzwanzig Handzeichnungen]\",\"date\":\"1919\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-standing-nude-funfundzwanzig-handzeichnungen\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FZrfjMOpcnf_UF1UgyKkkEA\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FZrfjMOpcnf_UF1UgyKkkEA\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1N2EyMDQyMDEzOWIyMTViMjUwMDE0YWE=\"}},{\"node\":{\"title\":\"Poppy Field [Gustav Klimt An Aftermath]\",\"date\":\"1931\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-poppy-field-gustav-klimt-an-aftermath\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002Frf-RnUsXd-iCgKDKO0eZyA\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002Frf-RnUsXd-iCgKDKO0eZyA\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1NjZmNDNlODljMThkYjZlYjcwMDAwM2Y=\"}},{\"node\":{\"title\":\"Water Serpents II [Das Werk Gustav Klimts]\",\"date\":\"1908-1914\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-water-serpents-ii-das-werk-gustav-klimts\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FFw7XQP7nr8PrgUrpkNFMZQ\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FFw7XQP7nr8PrgUrpkNFMZQ\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1OTYxNGE3ZDc2MjJkZDRkNWQ2OTEwYTU=\"}},{\"node\":{\"title\":\"The Kiss\",\"date\":\"1982\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":{\"__typename\":\"Money\",\"major\":75,\"currencyCode\":\"USD\"},\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-the-kiss\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FbA2VGfmX8NF3SLnffb1J0Q\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FbA2VGfmX8NF3SLnffb1J0Q\\u002Flarge.jpg\"},\"partner\":{\"name\":\"ArtWise\",\"href\":\"\\u002Fpartner\\u002Fartwise\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FMBbZxrPWHwsh0okl4CPFCg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FMBbZxrPWHwsh0okl4CPFCg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1OTI4NDY1OWNkNTMwZTVmODBjYTQwMjQ=\"},\"id\":\"UGFydG5lcjo1OTI4NDY1OWNkNTMwZTVmODBjYTQwMjM=\"},\"id\":\"QXJ0d29yazo1YTlmMDk0MDhiMGMxNDdlMGIyMDdkYjk=\"}},{\"node\":{\"title\":\"Girlfriends [Gustav Klimt An Aftermath]\",\"date\":\"1931\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-girlfriends-gustav-klimt-an-aftermath\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FG7W7-wcy_ZXWpnKCllriww\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FG7W7-wcy_ZXWpnKCllriww\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1NjZmNDRmOThiMGMxNDFkMzYwMDAwM2Y=\"}}]},\"id\":\"QXJ0aXN0OjRkOGI5MmI2NGViNjhhMWIyYzAwMDQxNA==\",\"internalID\":\"4d8b92b64eb68a1b2c000414\",\"isFollowed\":false,\"insightsList\":[{\"kind\":\"ACTIVE_SECONDARY_MARKET\",\"label\":\"Active Secondary Market\",\"entities\":[]}],\"auctionResultsConnection\":{\"edges\":[{\"node\":{\"priceRealized\":{\"display\":\"US$78.5m\"},\"organization\":\"Christie's\",\"sale_date\":\"2006\",\"id\":\"QXVjdGlvblJlc3VsdDoyNzA5ODkx\",\"price_realized\":{\"display\":\"US$78.5m\"}}}]},\"formattedNationalityAndBirthday\":\"Austrian, 1862–1918\"}}},\"data\":{\"artist\":{\"slug\":\"gustav-klimt\",\"statuses\":{\"shows\":true,\"cv\":true,\"articles\":true,\"artworks\":true,\"auctionLots\":true},\"counts\":{\"forSaleArtworks\":186,\"auctionResults\":1351,\"artworks\":247,\"follows\":20784},\"related\":{\"genes\":{\"edges\":[{\"node\":{\"slug\":\"vienna-secession\",\"id\":\"R2VuZTo0ZGU2ODcxZWFkNTA2ZTAwMDEwMDU0ZWI=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"symbolism\",\"id\":\"R2VuZTo0ZDkwZDE5M2RjZGQ1ZjQ0YTUwMDAwNzI=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"art-nouveau\",\"id\":\"R2VuZTo0ZDkwZDE4ZmRjZGQ1ZjQ0YTUwMDAwMmU=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"engaged-with-byzantine-and-medieval-art\",\"id\":\"R2VuZTo1NmMzMzc0ODY0Mjg3MDc5ODMwMDAyMzY=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"modern-and-impressionist-art\",\"id\":\"R2VuZTo1MDM1NjU3NGFiNzQ5ODAwMDIwMDAwMDY=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"austria\",\"id\":\"R2VuZTo1MTgyYmVhMzRiNzJjOGYyMjMwMDAwNWQ=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"mixed-media\",\"id\":\"R2VuZTo0ZDkwZDE4ZWRjZGQ1ZjQ0YTUwMDAwMGY=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"work-on-paper\",\"id\":\"R2VuZTo0ZDkwZDE4ZmRjZGQ1ZjQ0YTUwMDAwMTY=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"painting\",\"id\":\"R2VuZTo0ZDkwZDE4ZWRjZGQ1ZjQ0YTUwMDAwMTA=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"patterns\",\"id\":\"R2VuZTo0ZTVmZmViZTFiYjkxNzAwMDEwMDE1NjY=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"individual-portrait\",\"id\":\"R2VuZTo0ZTU3MDhjYjVkY2ExNDAwMDEwMDNhYTM=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"erotic\",\"id\":\"R2VuZTo0ZGNkZjM0ZjgwODBhZDAwMDEwMDNhODM=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"human-figure\",\"id\":\"R2VuZTo0ZDkwZDE5M2RjZGQ1ZjQ0YTUwMDAwN2M=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"nude\",\"id\":\"R2VuZTo0ZDkzOTZkYjE3Y2IxMzI1MzcwMDA0ODA=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"landscapes\",\"id\":\"R2VuZTo0ZDkwZDE5M2RjZGQ1ZjQ0YTUwMDAwN2Q=\",\"__typename\":\"Gene\"}},{\"node\":{\"slug\":\"flora\",\"id\":\"R2VuZTo0ZDkwZDE5NmRjZGQ1ZjQ0YTUwMDAwY2I=\",\"__typename\":\"Gene\"}}]}},\"highlights\":{\"artistPartnersConnection\":{\"edges\":[]},\"partnersConnection\":{\"edges\":[]}},\"insights\":[{\"type\":\"SOLO_SHOW\",\"__typename\":\"ArtistInsight\"},{\"type\":\"GROUP_SHOW\",\"__typename\":\"ArtistInsight\"},{\"type\":\"COLLECTED\",\"__typename\":\"ArtistInsight\"},{\"type\":\"REVIEWED\",\"__typename\":\"ArtistInsight\"},{\"type\":\"BIENNIAL\",\"__typename\":\"ArtistInsight\"},{\"type\":\"ACTIVE_SECONDARY_MARKET\",\"__typename\":\"ArtistInsight\"}],\"biographyBlurb\":{\"text\":\"\\u003Cp\\u003EGustav Klimt is best known for his opulently gilded, Byzantine-inspired \\u003Ca href=\\\"https:\\u002F\\u002Fwww.artsy.net\\u002Fgene\\u002Fart-nouveau\\\"\\u003Eart nouveau\\u003C\\u002Fa\\u003E  \\u003Ca href=\\\"https:\\u002F\\u002Fwww.artsy.net\\u002Fcollection\\u002Fgustav-klimt-portraits\\\"\\u003Eportraits\\u003C\\u002Fa\\u003E of women, which epitomize fin de siècle (and early 20th-century) Vienna. His most famous paintings, \\u003Cem\\u003EPortrait of Adele Bloch-Bauer\\u003C\\u002Fem\\u003E (1907) and \\u003Cem\\u003E\\u003Ca href=\\\"https:\\u002F\\u002Fwww.artsy.net\\u002Fartwork\\u002Fgustav-klimt-the-kiss\\\"\\u003EThe Kiss\\u003C\\u002Fa\\u003E\\u003C\\u002Fem\\u003E (1907), are among the most recognizable works of the early 20th century. Klimt’s \\u003Ca href=\\\"https:\\u002F\\u002Fwww.artsy.net\\u002Fgene\\u002Fsymbolism\\\"\\u003ESymbolist\\u003C\\u002Fa\\u003E pastiches of pale nudes, allegorical gardens, and erotic subject matter celebrated the intersection of fine and decorative art and became exemplars of the \\u003Ca href=\\\"http:\\u002F\\u002Fartsy.net\\u002Fgene\\u002Fvienna-secession\\\"\\u003EVienna Secession\\u003C\\u002Fa\\u003E. Today, his paintings, drawings, and objets belong in collections worldwide, including the Neue Galerie and the \\u003Ca href=\\\"https:\\u002F\\u002Fwww.artsy.net\\u002Fpartner\\u002Fthe-museum-of-modern-art\\\"\\u003EMuseum of Modern Art\\u003C\\u002Fa\\u003E in New York, and the Leopold Museum and Belvedere Museum and Palace in Vienna. On the secondary market, Klimt’s work routinely sells for millions. While Klimt’s narrative paintings explore psychosexual anxiety and the human condition, his vibrant landscapes demonstrate a keen engagement with light and the abstract patterns of nature. \\u003C\\u002Fp\\u003E\\n\",\"credit\":null,\"partnerID\":null},\"name\":\"Gustav Klimt\",\"nationality\":\"Austrian\",\"birthday\":\"1862\",\"deathday\":\"1918\",\"gender\":\"male\",\"href\":\"\\u002Fartist\\u002Fgustav-klimt\",\"meta\":{\"description\":\"Find the latest shows, biography, and artworks for sale by Gustav Klimt. Gustav Klimt is best known for his opulently gilded, Byzantine-inspired art nouveau …\"},\"alternate_names\":[],\"image\":{\"versions\":[\"four_thirds\",\"large\",\"square\",\"tall\"],\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002Fn3RTu8EOaXrq3UCBXjh1Ag\\u002Flarge.jpg\",\"square\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002Fn3RTu8EOaXrq3UCBXjh1Ag\\u002Fsquare.jpg\",\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fn3RTu8EOaXrq3UCBXjh1Ag%2Flarge.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fn3RTu8EOaXrq3UCBXjh1Ag%2Flarge.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fn3RTu8EOaXrq3UCBXjh1Ag%2Flarge.jpg 2x\"}},\"blurb\":\"Gustav Klimt is best known for his opulently gilded, Byzantine-inspired [art nouveau](https:\\u002F\\u002Fwww.artsy.net\\u002Fgene\\u002Fart-nouveau)  [portraits](https:\\u002F\\u002Fwww.artsy.net\\u002Fcollection\\u002Fgustav-klimt-portraits) of women, which epitomize fin de siècle (and early 20th-century) Vienna. His most famous paintings, _Portrait of Adele Bloch-Bauer_ (1907) and _[The Kiss](https:\\u002F\\u002Fwww.artsy.net\\u002Fartwork\\u002Fgustav-klimt-the-kiss)_ (1907), are among the most recognizable works of the early 20th century. Klimt’s [Symbolist](https:\\u002F\\u002Fwww.artsy.net\\u002Fgene\\u002Fsymbolism) pastiches of pale nudes, allegorical gardens, and erotic subject matter celebrated the intersection of fine and decorative art and became exemplars of the [Vienna Secession](http:\\u002F\\u002Fartsy.net\\u002Fgene\\u002Fvienna-secession). Today, his paintings, drawings, and objets belong in collections worldwide, including the Neue Galerie and the [Museum of Modern Art](https:\\u002F\\u002Fwww.artsy.net\\u002Fpartner\\u002Fthe-museum-of-modern-art) in New York, and the Leopold Museum and Belvedere Museum and Palace in Vienna. On the secondary market, Klimt’s work routinely sells for millions. While Klimt’s narrative paintings explore psychosexual anxiety and the human condition, his vibrant landscapes demonstrate a keen engagement with light and the abstract patterns of nature. \",\"artworks_connection\":{\"edges\":[{\"node\":{\"title\":\"Danaë [Das Werk Gustav Klimts]\",\"date\":\"1908-1914\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-danae-das-werk-gustav-klimts\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FStnM65TItGlWf869shn_lw\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FStnM65TItGlWf869shn_lw\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1OTYxNGE3Y2M5ZGMyNDZlYTBjOWE1ZjM=\"}},{\"node\":{\"title\":\"Judith [Das Werk Gustav Klimts]\",\"date\":\"1908-1914\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-judith-das-werk-gustav-klimts\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002F66PZrg542AFfN1LCo2JMvQ\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002F66PZrg542AFfN1LCo2JMvQ\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1Nzc3ZGMyNWNkNTMwZTY1ZjEwMDAxZjQ=\"}},{\"node\":{\"title\":\"Malcesine on Lake Garda [Gustav Klimt An Aftermath]\",\"date\":\"1931\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-malcesine-on-lake-garda-gustav-klimt-an-aftermath\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FlUJcL_7hQd8czIMBDLLK3Q\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FlUJcL_7hQd8czIMBDLLK3Q\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1NjZmNDQ5YzRjMGFmNTEzZDMwMDAwMzI=\"}},{\"node\":{\"title\":\"The Virgin [Das Werk Gustav Klimts]\",\"date\":\"1908-1914\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-the-virgin-das-werk-gustav-klimts\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FYme8vYIcH2W1BA_jqx0HhQ\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FYme8vYIcH2W1BA_jqx0HhQ\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1OTYxNGE3ZGIyMDJhMzRkNjY1MzIxYzE=\"}},{\"node\":{\"title\":\"Sunflowers [Gustav Klimt An Aftermath]\",\"date\":\"1931\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-sunflowers-gustav-klimt-an-aftermath\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FqlOeKMCvHLo_QtyznsR7Ug\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FqlOeKMCvHLo_QtyznsR7Ug\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1NjZmNDNmODc2MTQzZjdhNGUwMDAwMjA=\"}},{\"node\":{\"title\":\"Standing Nude [Fünfundzwanzig Handzeichnungen]\",\"date\":\"1919\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-standing-nude-funfundzwanzig-handzeichnungen\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FZrfjMOpcnf_UF1UgyKkkEA\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FZrfjMOpcnf_UF1UgyKkkEA\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1N2EyMDQyMDEzOWIyMTViMjUwMDE0YWE=\"}},{\"node\":{\"title\":\"Poppy Field [Gustav Klimt An Aftermath]\",\"date\":\"1931\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-poppy-field-gustav-klimt-an-aftermath\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002Frf-RnUsXd-iCgKDKO0eZyA\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002Frf-RnUsXd-iCgKDKO0eZyA\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1NjZmNDNlODljMThkYjZlYjcwMDAwM2Y=\"}},{\"node\":{\"title\":\"Water Serpents II [Das Werk Gustav Klimts]\",\"date\":\"1908-1914\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-water-serpents-ii-das-werk-gustav-klimts\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FFw7XQP7nr8PrgUrpkNFMZQ\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FFw7XQP7nr8PrgUrpkNFMZQ\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1OTYxNGE3ZDc2MjJkZDRkNWQ2OTEwYTU=\"}},{\"node\":{\"title\":\"The Kiss\",\"date\":\"1982\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":{\"__typename\":\"Money\",\"major\":75,\"currencyCode\":\"USD\"},\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-the-kiss\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FbA2VGfmX8NF3SLnffb1J0Q\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FbA2VGfmX8NF3SLnffb1J0Q\\u002Flarge.jpg\"},\"partner\":{\"name\":\"ArtWise\",\"href\":\"\\u002Fpartner\\u002Fartwise\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FMBbZxrPWHwsh0okl4CPFCg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FMBbZxrPWHwsh0okl4CPFCg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1OTI4NDY1OWNkNTMwZTVmODBjYTQwMjQ=\"},\"id\":\"UGFydG5lcjo1OTI4NDY1OWNkNTMwZTVmODBjYTQwMjM=\"},\"id\":\"QXJ0d29yazo1YTlmMDk0MDhiMGMxNDdlMGIyMDdkYjk=\"}},{\"node\":{\"title\":\"Girlfriends [Gustav Klimt An Aftermath]\",\"date\":\"1931\",\"description\":null,\"category\":\"Print\",\"price_currency\":\"USD\",\"listPrice\":null,\"availability\":\"for sale\",\"href\":\"\\u002Fartwork\\u002Fgustav-klimt-girlfriends-gustav-klimt-an-aftermath\",\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FG7W7-wcy_ZXWpnKCllriww\\u002Fsmall.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FG7W7-wcy_ZXWpnKCllriww\\u002Flarge.jpg\"},\"partner\":{\"name\":\"Jason Jacques Gallery\",\"href\":\"\\u002Fpartner\\u002Fjason-jacques-gallery\",\"profile\":{\"image\":{\"small\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\",\"large\":\"https:\\u002F\\u002Fd32dm0rphc51dk.cloudfront.net\\u002FWu1yOCLX_umGyFWP4VSpsg\\u002Fsquare.jpg\"},\"id\":\"UHJvZmlsZTo1MTU5ZGFhYTlhNjA4MzI0MzkwMDAxNjc=\"},\"id\":\"UGFydG5lcjo1MDYzNjQ3MTdmYWQzNzAwMDIwMDA0MGE=\"},\"id\":\"QXJ0d29yazo1NjZmNDRmOThiMGMxNDFkMzYwMDAwM2Y=\"}}]},\"id\":\"QXJ0aXN0OjRkOGI5MmI2NGViNjhhMWIyYzAwMDQxNA==\",\"internalID\":\"4d8b92b64eb68a1b2c000414\",\"isFollowed\":false,\"insightsList\":[{\"kind\":\"ACTIVE_SECONDARY_MARKET\",\"label\":\"Active Secondary Market\",\"entities\":[]}],\"auctionResultsConnection\":{\"edges\":[{\"node\":{\"priceRealized\":{\"display\":\"US$78.5m\"},\"organization\":\"Christie's\",\"sale_date\":\"2006\",\"id\":\"QXVjdGlvblJlc3VsdDoyNzA5ODkx\",\"price_realized\":{\"display\":\"US$78.5m\"}}}]},\"formattedNationalityAndBirthday\":\"Austrian, 1862–1918\"}}}],[\"{\\\"queryID\\\":\\\"artistRoutes_AuctionResultsQuery\\\",\\\"variables\\\":{\\\"allowEmptyCreatedDates\\\":true,\\\"artistID\\\":\\\"gustav-klimt\\\",\\\"categories\\\":[],\\\"createdAfterYear\\\":0,\\\"createdBeforeYear\\\":10000,\\\"organizations\\\":[],\\\"sizes\\\":[]}}\",{\"ok\":true,\"status\":200,\"url\":\"https:\\u002F\\u002Fmetaphysics-production.artsy.net\\u002Fv2\",\"headers\":{\"_headers\":{\"date\":[\"Fri, 29 Jul 2022 20:03:31 GMT\"],\"content-type\":[\"application\\u002Fjson; charset=utf-8\"],\"transfer-encoding\":[\"chunked\"],\"connection\":[\"close\"],\"x-powered-by\":[\"Express\"],\"access-control-allow-origin\":[\"*\"],\"etag\":[\"W\\u002F\\\"2e31-J4tvWukKyFVH1W+DuA01hoF5hAI\\\"\"],\"vary\":[\"Accept-Encoding\"],\"content-encoding\":[\"gzip\"],\"strict-transport-security\":[\"max-age=15724800; includeSubDomains\"]}},\"json\":{\"data\":{\"artist\":{\"slug\":\"gustav-klimt\",\"internalID\":\"4d8b92b64eb68a1b2c000414\",\"name\":\"Gustav Klimt\",\"auctionResultsConnection\":{\"createdYearRange\":{\"startAt\":1862,\"endAt\":1946},\"pageInfo\":{\"hasNextPage\":true,\"endCursor\":\"YXJyYXljb25uZWN0aW9uOjk=\"},\"pageCursors\":{\"around\":[{\"cursor\":\"YXJyYXljb25uZWN0aW9uOi0x\",\"page\":1,\"isCurrent\":true},{\"cursor\":\"YXJyYXljb25uZWN0aW9uOjk=\",\"page\":2,\"isCurrent\":false},{\"cursor\":\"YXJyYXljb25uZWN0aW9uOjE5\",\"page\":3,\"isCurrent\":false},{\"cursor\":\"YXJyYXljb25uZWN0aW9uOjI5\",\"page\":4,\"isCurrent\":false}],\"first\":null,\"last\":{\"cursor\":\"YXJyYXljb25uZWN0aW9uOjk4OQ==\",\"page\":100,\"isCurrent\":false},\"previous\":null},\"totalCount\":1351,\"edges\":[{\"node\":{\"title\":\"Zusammengekauerter und nach rechts stehender Frauenakt (Crouched Female Nude Standing to the Right)\",\"dimension_text\":\"447 by 307 mm\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FwiULzirR6CJs0SLWADxDFg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FwiULzirR6CJs0SLWADxDFg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FwiULzirR6CJs0SLWADxDFg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"black chalk on paper;\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1902\",\"saleDate\":\"2022-07-06T09:00:00Z\",\"boughtIn\":false,\"currency\":\"GBP\",\"price_realized\":{\"display\":\"£85,680\",\"display_usd\":\"US$102,004\",\"cents_usd\":10200376},\"performance\":{\"mid\":\"172%\"},\"estimate\":{\"display\":\"£28,000–£35,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2OTUwODQy\"}},{\"node\":{\"title\":\"”Zwei Studien eines mit hochgezogenem rechten Knie sitzenden Aktes” (Fakultätsbild Medizin)\",\"dimension_text\":\"31.0 x 41.2 cm\",\"organization\":\"Grisebach\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FjSrUGVFucCCKvl04v7rhqw%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FjSrUGVFucCCKvl04v7rhqw%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FjSrUGVFucCCKvl04v7rhqw%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"Chalk on buff-coloured paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1901\",\"saleDate\":\"2022-06-03T00:00:00Z\",\"boughtIn\":false,\"currency\":\"EUR\",\"price_realized\":{\"display\":\"€11,250\",\"display_usd\":\"US$12,095\",\"cents_usd\":1209547},\"performance\":{\"mid\":\"-10%\"},\"estimate\":{\"display\":\"€10,000–€15,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODc3MTky\"}},{\"node\":{\"title\":\"Mit Strümpfen, die rechte Hand an der Schulter\",\"dimension_text\":\"29.0 x 56.0 cm\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F6EAIqqyjP2whBlg0u5fsMg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F6EAIqqyjP2whBlg0u5fsMg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F6EAIqqyjP2whBlg0u5fsMg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"pencil on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":null,\"saleDate\":\"2022-05-18T00:00:00Z\",\"boughtIn\":false,\"currency\":\"USD\",\"price_realized\":{\"display\":\"US$56,700\",\"display_usd\":\"US$56,700\",\"cents_usd\":5670000},\"performance\":{\"mid\":\"89%\"},\"estimate\":{\"display\":\"US$25,000–US$35,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODc0OTE1\"}},{\"node\":{\"title\":\"Stehend von vorne, die Rechte abgestützt (Bildnis Paula Zuckerkandl) (Standing from the Front, the Right Hand Supported (Portrait of Paula Zuckerkandl))\",\"dimension_text\":\"37.5 x 57.0 cm\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FndM-A54W-RpcV0JVx6Z8Qg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FndM-A54W-RpcV0JVx6Z8Qg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FndM-A54W-RpcV0JVx6Z8Qg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"pencil on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1911\",\"saleDate\":\"2022-04-28T00:00:00Z\",\"boughtIn\":false,\"currency\":\"GBP\",\"price_realized\":{\"display\":\"£60,480\",\"display_usd\":\"US$75,841\",\"cents_usd\":7584089},\"performance\":{\"mid\":\"51%\"},\"estimate\":{\"display\":\"£30,000–£50,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODczNjc0\"}},{\"node\":{\"title\":\"Auf dem Bauch liegender Akt mit Pelzboa (Reclining Nude with Fur Boa)\",\"dimension_text\":\"37.5 by 56.5 cm.\\n14¾ by 22¼ in.\\n52.5 by 72 cm.\\n20½ by 28¼ in.\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fltv2bqwpZuK54fwtj2Hlvw%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fltv2bqwpZuK54fwtj2Hlvw%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fltv2bqwpZuK54fwtj2Hlvw%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"pencil on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1904\",\"saleDate\":\"2022-04-07T12:00:00Z\",\"boughtIn\":false,\"currency\":\"EUR\",\"price_realized\":{\"display\":\"€75,600\",\"display_usd\":\"US$82,392\",\"cents_usd\":8239189},\"performance\":{\"mid\":\"26%\"},\"estimate\":{\"display\":\"€50,000–€70,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODA5Mjg1\"}},{\"node\":{\"title\":\"Studie für 'Danaë', mit überschlagenen Beinen sitzender Akt nach rechts\",\"dimension_text\":\"31.0 x 46.6 cm\",\"organization\":\"Bonhams\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FtPTegGliMZ45yKAQI2Z6bg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FtPTegGliMZ45yKAQI2Z6bg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FtPTegGliMZ45yKAQI2Z6bg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"brown crayon on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1903\",\"saleDate\":\"2022-04-07T00:00:00Z\",\"boughtIn\":false,\"currency\":\"GBP\",\"price_realized\":{\"display\":\"£60,780\",\"display_usd\":\"US$79,435\",\"cents_usd\":7943530},\"performance\":{\"mid\":\"183%\"},\"estimate\":{\"display\":\"£18,000–£25,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODA5NTQx\"}},{\"node\":{\"title\":\"Auf dem Bauch liegender Akt mit Pelzboa (Reclining Nude with Fur Boa)\",\"dimension_text\":\"56.5 x 37.5 cm\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FLKRChsE04FI-FOAV-XHCRg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FLKRChsE04FI-FOAV-XHCRg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FLKRChsE04FI-FOAV-XHCRg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"pencil on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1904\",\"saleDate\":\"2022-04-01T00:00:00Z\",\"boughtIn\":false,\"currency\":\"EUR\",\"price_realized\":{\"display\":\"€75,600\",\"display_usd\":\"US$83,714\",\"cents_usd\":8371407},\"performance\":{\"mid\":\"26%\"},\"estimate\":{\"display\":\"€50,000–€70,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODA4MTc2\"}},{\"node\":{\"title\":\"Hockender Weiblicher Akt (Squatting Female Nude)\",\"dimension_text\":\"17¾ by 12⅝ in.\\n45 by 32.1 cm.\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F7lEBuZpLlz8UWl6bW5sFeg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F7lEBuZpLlz8UWl6bW5sFeg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F7lEBuZpLlz8UWl6bW5sFeg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"black chalk on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1901\",\"saleDate\":\"2022-03-17T16:00:00Z\",\"boughtIn\":false,\"currency\":\"USD\",\"price_realized\":{\"display\":\"US$37,800\",\"display_usd\":\"US$37,800\",\"cents_usd\":3780000},\"performance\":{\"mid\":\"89%\"},\"estimate\":{\"display\":\"US$15,000–US$25,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2Njk4Njcx\"}},{\"node\":{\"title\":\"Umarmendes Paar\",\"dimension_text\":\"37.1 x 54.2 cm\",\"organization\":\"Swann\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FpVIDSUOUD98q5j1qaGsolw%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FpVIDSUOUD98q5j1qaGsolw%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FpVIDSUOUD98q5j1qaGsolw%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"Pencil on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1907\",\"saleDate\":\"2022-03-10T00:00:00Z\",\"boughtIn\":false,\"currency\":\"USD\",\"price_realized\":{\"display\":\"US$87,500\",\"display_usd\":\"US$87,500\",\"cents_usd\":8750000},\"performance\":{\"mid\":\"119%\"},\"estimate\":{\"display\":\"US$30,000–US$50,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2NzAzNjg0\"}},{\"node\":{\"title\":\"Stehender weiblicher Akt, Hände hinter dem Rücken verschränkt\",\"dimension_text\":\"37.0 x 56.0 cm\",\"organization\":\"Swann\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Ft_uJuZKzSW5ck11ZuovEmQ%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Ft_uJuZKzSW5ck11ZuovEmQ%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Ft_uJuZKzSW5ck11ZuovEmQ%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"Reddish orange crayon on cream wove paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1906\",\"saleDate\":\"2022-03-10T00:00:00Z\",\"boughtIn\":false,\"currency\":\"USD\",\"price_realized\":{\"display\":\"US$27,500\",\"display_usd\":\"US$27,500\",\"cents_usd\":2750000},\"performance\":{\"mid\":\"-31%\"},\"estimate\":{\"display\":\"US$30,000–US$50,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2NzAzNjgz\"}}]},\"id\":\"QXJ0aXN0OjRkOGI5MmI2NGViNjhhMWIyYzAwMDQxNA==\"}}},\"data\":{\"artist\":{\"slug\":\"gustav-klimt\",\"internalID\":\"4d8b92b64eb68a1b2c000414\",\"name\":\"Gustav Klimt\",\"auctionResultsConnection\":{\"createdYearRange\":{\"startAt\":1862,\"endAt\":1946},\"pageInfo\":{\"hasNextPage\":true,\"endCursor\":\"YXJyYXljb25uZWN0aW9uOjk=\"},\"pageCursors\":{\"around\":[{\"cursor\":\"YXJyYXljb25uZWN0aW9uOi0x\",\"page\":1,\"isCurrent\":true},{\"cursor\":\"YXJyYXljb25uZWN0aW9uOjk=\",\"page\":2,\"isCurrent\":false},{\"cursor\":\"YXJyYXljb25uZWN0aW9uOjE5\",\"page\":3,\"isCurrent\":false},{\"cursor\":\"YXJyYXljb25uZWN0aW9uOjI5\",\"page\":4,\"isCurrent\":false}],\"first\":null,\"last\":{\"cursor\":\"YXJyYXljb25uZWN0aW9uOjk4OQ==\",\"page\":100,\"isCurrent\":false},\"previous\":null},\"totalCount\":1351,\"edges\":[{\"node\":{\"title\":\"Zusammengekauerter und nach rechts stehender Frauenakt (Crouched Female Nude Standing to the Right)\",\"dimension_text\":\"447 by 307 mm\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FwiULzirR6CJs0SLWADxDFg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FwiULzirR6CJs0SLWADxDFg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FwiULzirR6CJs0SLWADxDFg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"black chalk on paper;\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1902\",\"saleDate\":\"2022-07-06T09:00:00Z\",\"boughtIn\":false,\"currency\":\"GBP\",\"price_realized\":{\"display\":\"£85,680\",\"display_usd\":\"US$102,004\",\"cents_usd\":10200376},\"performance\":{\"mid\":\"172%\"},\"estimate\":{\"display\":\"£28,000–£35,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2OTUwODQy\"}},{\"node\":{\"title\":\"”Zwei Studien eines mit hochgezogenem rechten Knie sitzenden Aktes” (Fakultätsbild Medizin)\",\"dimension_text\":\"31.0 x 41.2 cm\",\"organization\":\"Grisebach\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FjSrUGVFucCCKvl04v7rhqw%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FjSrUGVFucCCKvl04v7rhqw%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FjSrUGVFucCCKvl04v7rhqw%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"Chalk on buff-coloured paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1901\",\"saleDate\":\"2022-06-03T00:00:00Z\",\"boughtIn\":false,\"currency\":\"EUR\",\"price_realized\":{\"display\":\"€11,250\",\"display_usd\":\"US$12,095\",\"cents_usd\":1209547},\"performance\":{\"mid\":\"-10%\"},\"estimate\":{\"display\":\"€10,000–€15,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODc3MTky\"}},{\"node\":{\"title\":\"Mit Strümpfen, die rechte Hand an der Schulter\",\"dimension_text\":\"29.0 x 56.0 cm\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F6EAIqqyjP2whBlg0u5fsMg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F6EAIqqyjP2whBlg0u5fsMg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F6EAIqqyjP2whBlg0u5fsMg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"pencil on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":null,\"saleDate\":\"2022-05-18T00:00:00Z\",\"boughtIn\":false,\"currency\":\"USD\",\"price_realized\":{\"display\":\"US$56,700\",\"display_usd\":\"US$56,700\",\"cents_usd\":5670000},\"performance\":{\"mid\":\"89%\"},\"estimate\":{\"display\":\"US$25,000–US$35,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODc0OTE1\"}},{\"node\":{\"title\":\"Stehend von vorne, die Rechte abgestützt (Bildnis Paula Zuckerkandl) (Standing from the Front, the Right Hand Supported (Portrait of Paula Zuckerkandl))\",\"dimension_text\":\"37.5 x 57.0 cm\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FndM-A54W-RpcV0JVx6Z8Qg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FndM-A54W-RpcV0JVx6Z8Qg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FndM-A54W-RpcV0JVx6Z8Qg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"pencil on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1911\",\"saleDate\":\"2022-04-28T00:00:00Z\",\"boughtIn\":false,\"currency\":\"GBP\",\"price_realized\":{\"display\":\"£60,480\",\"display_usd\":\"US$75,841\",\"cents_usd\":7584089},\"performance\":{\"mid\":\"51%\"},\"estimate\":{\"display\":\"£30,000–£50,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODczNjc0\"}},{\"node\":{\"title\":\"Auf dem Bauch liegender Akt mit Pelzboa (Reclining Nude with Fur Boa)\",\"dimension_text\":\"37.5 by 56.5 cm.\\n14¾ by 22¼ in.\\n52.5 by 72 cm.\\n20½ by 28¼ in.\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fltv2bqwpZuK54fwtj2Hlvw%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fltv2bqwpZuK54fwtj2Hlvw%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Fltv2bqwpZuK54fwtj2Hlvw%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"pencil on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1904\",\"saleDate\":\"2022-04-07T12:00:00Z\",\"boughtIn\":false,\"currency\":\"EUR\",\"price_realized\":{\"display\":\"€75,600\",\"display_usd\":\"US$82,392\",\"cents_usd\":8239189},\"performance\":{\"mid\":\"26%\"},\"estimate\":{\"display\":\"€50,000–€70,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODA5Mjg1\"}},{\"node\":{\"title\":\"Studie für 'Danaë', mit überschlagenen Beinen sitzender Akt nach rechts\",\"dimension_text\":\"31.0 x 46.6 cm\",\"organization\":\"Bonhams\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FtPTegGliMZ45yKAQI2Z6bg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FtPTegGliMZ45yKAQI2Z6bg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FtPTegGliMZ45yKAQI2Z6bg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"brown crayon on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1903\",\"saleDate\":\"2022-04-07T00:00:00Z\",\"boughtIn\":false,\"currency\":\"GBP\",\"price_realized\":{\"display\":\"£60,780\",\"display_usd\":\"US$79,435\",\"cents_usd\":7943530},\"performance\":{\"mid\":\"183%\"},\"estimate\":{\"display\":\"£18,000–£25,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODA5NTQx\"}},{\"node\":{\"title\":\"Auf dem Bauch liegender Akt mit Pelzboa (Reclining Nude with Fur Boa)\",\"dimension_text\":\"56.5 x 37.5 cm\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FLKRChsE04FI-FOAV-XHCRg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FLKRChsE04FI-FOAV-XHCRg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FLKRChsE04FI-FOAV-XHCRg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"pencil on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1904\",\"saleDate\":\"2022-04-01T00:00:00Z\",\"boughtIn\":false,\"currency\":\"EUR\",\"price_realized\":{\"display\":\"€75,600\",\"display_usd\":\"US$83,714\",\"cents_usd\":8371407},\"performance\":{\"mid\":\"26%\"},\"estimate\":{\"display\":\"€50,000–€70,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2ODA4MTc2\"}},{\"node\":{\"title\":\"Hockender Weiblicher Akt (Squatting Female Nude)\",\"dimension_text\":\"17¾ by 12⅝ in.\\n45 by 32.1 cm.\",\"organization\":\"Sotheby's\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F7lEBuZpLlz8UWl6bW5sFeg%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F7lEBuZpLlz8UWl6bW5sFeg%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2F7lEBuZpLlz8UWl6bW5sFeg%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"black chalk on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1901\",\"saleDate\":\"2022-03-17T16:00:00Z\",\"boughtIn\":false,\"currency\":\"USD\",\"price_realized\":{\"display\":\"US$37,800\",\"display_usd\":\"US$37,800\",\"cents_usd\":3780000},\"performance\":{\"mid\":\"89%\"},\"estimate\":{\"display\":\"US$15,000–US$25,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2Njk4Njcx\"}},{\"node\":{\"title\":\"Umarmendes Paar\",\"dimension_text\":\"37.1 x 54.2 cm\",\"organization\":\"Swann\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FpVIDSUOUD98q5j1qaGsolw%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FpVIDSUOUD98q5j1qaGsolw%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FpVIDSUOUD98q5j1qaGsolw%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"Pencil on paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1907\",\"saleDate\":\"2022-03-10T00:00:00Z\",\"boughtIn\":false,\"currency\":\"USD\",\"price_realized\":{\"display\":\"US$87,500\",\"display_usd\":\"US$87,500\",\"cents_usd\":8750000},\"performance\":{\"mid\":\"119%\"},\"estimate\":{\"display\":\"US$30,000–US$50,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2NzAzNjg0\"}},{\"node\":{\"title\":\"Stehender weiblicher Akt, Hände hinter dem Rücken verschränkt\",\"dimension_text\":\"37.0 x 56.0 cm\",\"organization\":\"Swann\",\"images\":{\"larger\":{\"cropped\":{\"src\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Ft_uJuZKzSW5ck11ZuovEmQ%2Flarger.jpg\",\"srcSet\":\"https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Ft_uJuZKzSW5ck11ZuovEmQ%2Flarger.jpg 1x, https:\\u002F\\u002Fd7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2Ft_uJuZKzSW5ck11ZuovEmQ%2Flarger.jpg 2x\",\"width\":100,\"height\":100}}},\"mediumText\":\"Reddish orange crayon on cream wove paper\",\"categoryText\":\"Work on Paper\",\"date_text\":\"1906\",\"saleDate\":\"2022-03-10T00:00:00Z\",\"boughtIn\":false,\"currency\":\"USD\",\"price_realized\":{\"display\":\"US$27,500\",\"display_usd\":\"US$27,500\",\"cents_usd\":2750000},\"performance\":{\"mid\":\"-31%\"},\"estimate\":{\"display\":\"US$30,000–US$50,000\"},\"id\":\"QXVjdGlvblJlc3VsdDo2NzAzNjgz\"}}]},\"id\":\"QXJ0aXN0OjRkOGI5MmI2NGViNjhhMWIyYzAwMDQxNA==\"}}}]]"

auction = json.loads(string_)

In [170]:
auction

[['{"queryID":"artistRoutes_TopLevelQuery","variables":{"artistID":"gustav-klimt"}}',
  {'ok': True,
   'status': 200,
   'url': 'https://metaphysics-production.artsy.net/v2',
   'headers': {'_headers': {'date': ['Fri, 29 Jul 2022 20:03:31 GMT'],
     'content-type': ['application/json; charset=utf-8'],
     'transfer-encoding': ['chunked'],
     'connection': ['close'],
     'x-powered-by': ['Express'],
     'access-control-allow-origin': ['*'],
     'etag': ['W/"3af0-pe6whua1xs11KaR6d/2W+Qhz830"'],
     'vary': ['Accept-Encoding'],
     'content-encoding': ['gzip'],
     'strict-transport-security': ['max-age=15724800; includeSubDomains']}},
   'json': {'data': {'artist': {'slug': 'gustav-klimt',
      'statuses': {'shows': True,
       'cv': True,
       'articles': True,
       'artworks': True,
       'auctionLots': True},
      'counts': {'forSaleArtworks': 186,
       'auctionResults': 1351,
       'artworks': 247,
       'follows': 20784},
      'related': {'genes': {'edges': [

In [171]:
i = 0
auction_result = auction[1][1]['data']['artist']['auctionResultsConnection']['edges'][i]['node']
auction_result

{'title': 'Zusammengekauerter und nach rechts stehender Frauenakt (Crouched Female Nude Standing to the Right)',
 'dimension_text': '447 by 307 mm',
 'organization': "Sotheby's",
 'images': {'larger': {'cropped': {'src': 'https://d7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FwiULzirR6CJs0SLWADxDFg%2Flarger.jpg',
    'srcSet': 'https://d7hftxdivxxvm.cloudfront.net?resize_to=fill&width=100&height=100&quality=80&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FwiULzirR6CJs0SLWADxDFg%2Flarger.jpg 1x, https://d7hftxdivxxvm.cloudfront.net?resize_to=fill&width=200&height=200&quality=50&src=https%3A%2F%2Fd32dm0rphc51dk.cloudfront.net%2FwiULzirR6CJs0SLWADxDFg%2Flarger.jpg 2x',
    'width': 100,
    'height': 100}}},
 'mediumText': 'black chalk on paper;',
 'categoryText': 'Work on Paper',
 'date_text': '1902',
 'saleDate': '2022-07-06T09:00:00Z',
 'boughtIn': False,
 'currency': 'GBP',
 'price_realized': {'display'